In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime as dt
from sklearn.metrics import *

In [2]:
games = pd.read_csv('input/games.csv')

In [3]:
teams = pd.read_csv('input/teams.csv')

In [4]:
players = pd.read_csv('input/players.csv')

In [5]:
ranking = pd.read_csv('input/ranking.csv')

In [6]:
salaries = pd.read_csv('input/salaries.csv')

In [7]:
games = games.drop(columns = ["TEAM_ID_home","TEAM_ID_away","GAME_STATUS_TEXT"])

In [8]:
games = games.rename(columns={'GAME_DATE_EST':'FECHA',
                              'HOME_TEAM_ID':'EQUIPO_LOCAL_ID',
                              'VISITOR_TEAM_ID':'EQUIPO_VISITANTE_ID',
                              'SEASON':'TEMPORADA',
                              'PTS_home':'PTS_LOCAL',
                              'FG_PCT_home':'%TC_LOCAL',
                              'FT_PCT_home':'%TL_LOCAL',
                              'FG3_PCT_home':'%3P_LOCAL',
                              'AST_home':'AST_LOCAL',
                              'REB_home':'REB_LOCAL',
                              'PTS_away':'PTS_VISITANTE',
                              'FG_PCT_away':'%TC_VISITANTE',
                              'FT_PCT_away':'%TL_VISITANTE',
                              'FG3_PCT_away':'%3P_VISITANTE',
                              'AST_away':'AST_VISITANTE',
                              'REB_away':'REB_VISITANTE',
                              'HOME_TEAM_WINS':'VICTORIA'})

In [9]:
games.head()

,FECHA,GAME_ID,EQUIPO_LOCAL_ID,EQUIPO_VISITANTE_ID,TEMPORADA,PTS_LOCAL,%TC_LOCAL,%TL_LOCAL,%3P_LOCAL,AST_LOCAL,REB_LOCAL,PTS_VISITANTE,%TC_VISITANTE,%TL_VISITANTE,%3P_VISITANTE,AST_VISITANTE,REB_VISITANTE,VICTORIA
0,2020-03-01,21900895,1610612766,1610612749,2019,85.0,0.354,0.900,0.229,22.0,47.0,93.0,0.402,0.762,0.226,20.0,61.0,0
1,2020-03-01,21900896,1610612750,1610612742,2019,91.0,0.364,0.400,0.310,19.0,57.0,111.0,0.468,0.632,0.275,28.0,56.0,0
2,2020-03-01,21900897,1610612746,1610612755,2019,136.0,0.592,0.805,0.542,25.0,37.0,130.0,0.505,0.650,0.488,27.0,37.0,1
3,2020-03-01,21900898,1610612743,1610612761,2019,133.0,0.566,0.700,0.500,38.0,41.0,118.0,0.461,0.897,0.263,24.0,36.0,1
4,2020-03-01,21900899,1610612758,1610612765,2019,106.0,0.407,0.885,0.257,18.0,51.0,100.0,0.413,0.667,0.429,23.0,42.0,1


In [10]:
categoricalColumns = ['EQUIPO_LOCAL_ID', 'EQUIPO_VISITANTE_ID', 'GAME_ID','VICTORIA']
for i in categoricalColumns:
    games[i] = games[i].astype('category')


In [11]:
#games = pd.get_dummies(games, prefix=['VICTORIA'], columns=['VICTORIA'])

In [12]:
games['VICTORIA']= games['VICTORIA'].replace({0: 'VISITANTE', 1: 'LOCAL'})

In [13]:
games.to_csv('output/gamesClean.csv')

In [14]:
games.columns

Index(['FECHA', 'GAME_ID', 'EQUIPO_LOCAL_ID', 'EQUIPO_VISITANTE_ID',
       'TEMPORADA', 'PTS_LOCAL', '%TC_LOCAL', '%TL_LOCAL', '%3P_LOCAL',
       'AST_LOCAL', 'REB_LOCAL', 'PTS_VISITANTE', '%TC_VISITANTE',
       '%TL_VISITANTE', '%3P_VISITANTE', 'AST_VISITANTE', 'REB_VISITANTE',
       'VICTORIA'],
      dtype='object')

In [ ]:
#games = games.rename(columns={'VICTORIA_1':'VICTORIA_LOCAL',
#                             'VICTORIA_0':'VICTORIA_VISITANTE'
#                             })

In [ ]:
#len(games.VICTORIA_VISITANTE)

In [ ]:
#games['RESULTADO_V_L'] = games[['VICTORIA_VISITANTE','VICTORIA_LOCAL']].apply(lambda x: pd.Series([x.values]), axis=1)

In [ ]:
#games = games.drop(columns = ["VICTORIA_VISITANTE","VICTORIA_LOCAL"])

In [ ]:
#games.head()

In [ ]:
games = games.dropna()

In [ ]:
games.isnull().sum()

In [ ]:
games.dtypes

In [ ]:
games['FECHA'] = pd.to_datetime(games['FECHA'])
games['FECHA'] = games['FECHA'].apply(lambda x: x.toordinal())

In [ ]:
games.head()

In [ ]:
y = games.VICTORIA

In [ ]:
X = games.drop(columns =['VICTORIA','GAME_ID','EQUIPO_LOCAL_ID','EQUIPO_VISITANTE_ID','PTS_LOCAL','PTS_VISITANTE'])

In [ ]:
print(y.shape)
print(X.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.pipeline import make_pipeline
pipeline = [
    StandardScaler(),
    Normalizer(),
]

tr = make_pipeline(*pipeline)

Xpr = tr.fit_transform(X)
Xpr = pd.DataFrame(Xpr,columns=X.columns)
print(Xpr.info())
Xpr.head()

In [ ]:
from sklearn.model_selection import train_test_split

Xpr_train, Xpr_test, y_train, y_test = train_test_split(Xpr, y, test_size=0.2)

In [ ]:
print(Xpr_train.shape)
print(Xpr_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=800, max_depth=20, min_samples_leaf=3, random_state=111, n_jobs = -1)

model.fit(Xpr_train, y_train)

In [ ]:
y_pred=model.predict(Xpr_test)
print('Accurancy', accuracy_score(y_test, y_pred))
print("Precision", precision_score(y_test, y_pred, average='weighted'))
print("Recall", recall_score(y_test, y_pred, average='weighted'))

In [ ]:
model.predict_proba(Xpr_test)

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

y_score = model.predict_proba(Xpr_test)


display(pd.DataFrame({
    "real":y_pred,
    "predicted":y_test,
    "probaVisitante":y_score[:,1],
    "probaLocal":y_score[:,0]
}).head(30))


In [ ]:
games.to_json("output/games.json", orient="records")

In [ ]:
mongoimport --db nba --collection games --jsonArray --drop games.json